# Zadanie domowe 2
### Mateusz Sperkowski
W tym zadaniu korzystamy ze zbioru Allegro [https://www.dropbox.com/s/360xhh2d9lnaek3/allegro-api-transactions.csv?dl=1]

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv("allegro-api-transactions.csv", index_col = "lp")
print(data.columns)
cell2=len(data["it_location"].unique())
print(f"Unique number of it_locations: {cell2}")
data.head()

In [ ]:
data["it_location"] = data["it_location"].apply(str.lower)
c = len(data["it_location"].unique())
print(f"Unique number of it_locations after transformation: {c}")
data.head()

Należy zauważyć że w oryginalnych danych wartości "it_location" które powinny być takie same, przykładowo były napisane raz stylem wielbłądzim a raz wszystkie wielkimi literami. Zastosowałem przekształcenie wszystkich wyrazów do tylko małych liter co ograniczyło powtórzenia o 2 tysiące różnych wartości (miast). Przykład: "BIAŁYSTOK" i "Białystok".

## Kodowanie zmiennych kategorycznych

In [ ]:
import category_encoders as ce

In [ ]:
encoder = ce.TargetEncoder()
data['it_location'] = encoder.fit_transform(data['it_location'], data['price'])

In [ ]:
print(f"Unique number of it_locations after transformation: {len(data['it_location'].unique())}")
data["it_location"].head()

Target Encoding nie zwiększa wymiarowości problemu, co jest tu ważne gdy mamy prawie 8k miast, one hot by dodał 8k kolumn. Trzymając w jednej kolumnie te wartości można wyliczyć odległość między wartościami, co może i nie byłoby tak złe gdy są to miasta, jednak encoding ten oczywiście nie ma sposobu na takie określenie tego by to miało sens. Wiec odległość miedzy dwoma wartościami będzie bezsensowna informacją.

Target Encoding: Działa na bazie prawdopodobieństwa aposteriori (tzn. prawdopodobieństwo na podstawie doświadczenia losowego/zbioru testowego) i prawdopodobieństwa "oczekiwanego" dla zmiennej kategorycznej. Po kodowaniu tym zmniejszyła się ilość unikalnych wartości kolumny "it_location", co prawdopodobnie oznacza że przypisana została ta sama wartość dla różnych "it_location". Tak jakby kolumna kategoryczna została stransformowana do numerycznej. 

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
len(data["main_category"].unique())

### Onehot Encoding

In [ ]:
onehot = OneHotEncoder(handle_unknown='ignore')
encoded_onehot = pd.DataFrame(onehot.fit_transform(data[["main_category"]]).toarray())
data_onehot = data.join(encoded_onehot)
data_onehot.head(3)

Jest mało (27) unikalnych wartości w "main_category", co pozwala zapewnia że moglibyśmy spokojnie używać one hot encoding. Problemem byłoby gdyby występowało znacznie więcej różnych wartości.

### Ordinal Encoding

In [ ]:
ordinal = ce.ordinal.OrdinalEncoder()
encoded_ordinal = pd.DataFrame(ordinal.fit_transform(data[["main_category"]]))

In [ ]:
print(len(encoded_ordinal["main_category"].unique()))
encoded_ordinal.head()

Losowo przypisane są wartości całkowite do unikalnych wartości w "main_category".

### Count Encoding

In [ ]:
count = ce.count.CountEncoder(normalize=True)
encoded_count = pd.DataFrame(count.fit_transform(data[["main_category"]]))
encoded_count.head(3)

Zmiennej kategorycznej przypisana jest liczba jej częstości (tu dodatkowo znormalizowana do przedziału (0,1)). Więc w przypadku równych częstości różnych zmiennych, przypisane zostaną do tej samej wartości po encodingu.

## Uzupełnianie braków

 uzupełnić z użyciem jednego z automatycznych narzędzi: Nearest neighbors imputation lub Multivariate feature imputation (https://scikit-learn.org/stable/modules/impute.html).
 
Opisać wnioski z analizy jakości imputacji i umieścić podsumowujący wykres.

In [ ]:
import random
from sklearn.metrics import mean_squared_error
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
size = 210000
#size = 21000
random.seed(123456)

In [ ]:
res0 = []
print("Iteration: ", end="")
for i in range(10):
    data2 = data[["price", "it_seller_rating", "it_quantity"]]
    data2 = data2.iloc[0:size, :]
    ix = [row for row in range(data2.shape[0])]
    for row in random.sample(ix, int(round(.1*len(ix)))):
        data2.loc[row, "it_seller_rating"] = np.nan
    imputer = IterativeImputer(max_iter=10, random_state=0)
    data2 = imputer.fit_transform(data2)
    result = mean_squared_error(data.loc[0:(size-1),"it_seller_rating"], data2[:,1], squared=False)
    res0.append([i,result])
    print(i, end="")
print(", Finished")

In [ ]:
z = [val for (i, val) in res0]
print(f"it_seller_rating std: {np.std(z)}")
avg_z = np.mean(z)

In [ ]:
x=[0,1,2,3,4,5,6,7,8,9]
sns.barplot(x=x, y=z, color="tab:blue")
sns.lineplot(x=x, y=[avg_z]*10, color="red")
plt.title("Plot Of Error in iterations and mean error line")
plt.ylabel("RMSE")
plt.xlabel("Iterations")
plt.show()

In [ ]:
res2 = []
print("Iteration: ", end="")
for i in range(10):
    data2 = data[["price", "it_seller_rating", "it_quantity"]]
    data2 = data2.iloc[0:size, :]
    ix = [row for row in range(data2.shape[0])]
    for row in random.sample(ix, int(round(.1*len(ix)))):
        data2.loc[row, "it_seller_rating"] = np.nan
    for row in random.sample(ix, int(round(.1*len(ix)))):
        data2.loc[row, "it_quantity"] = np.nan
    imputer = IterativeImputer(max_iter=10, random_state=0)
    data2 = imputer.fit_transform(data2)
    result = mean_squared_error(data.loc[0:(size-1),"it_seller_rating"], data2[:,1], squared=False)
    result2 = mean_squared_error(data.loc[0:(size-1),"it_quantity"], data2[:,2], squared=False)
    res2.append([i,result, result2])
    print(i, end="")
print(", Finished")

In [ ]:
rate = []
quant = []
for (i, rating, quantity) in res2:
    rate.append(rating)
    quant.append(quantity)

rate_avg = np.mean(rate)
quant_avg = np.mean(quant)
print(f"it_seller_rating std: {np.std(rate)}")
print(f"it_quantity std: {np.std(quant)}")

In [ ]:
x=[0,1,2,3,4,5,6,7,8,9]
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize = (14,6))
sns.barplot(x=x, y=rate, color="tab:blue", ax = ax1)
sns.lineplot(x=x, y=[rate_avg]*10, color="red", ax= ax1)
sns.barplot(x=x, y=quant, color="tab:blue", ax = ax2)
sns.lineplot(x=x, y=[quant_avg]*10, color="red", ax= ax2)
plt.suptitle("Plot Of Error in iterations and mean error line")
ax1.title.set_text('Plot for it_seller_rating')
ax2.title.set_text('Plot for it_quantity')
ax1.set_xlabel('Iteration')
ax1.set_ylabel('RMSE')
ax2.set_xlabel('Iteration')
ax2.set_ylabel('RMSE')
plt.show()

### Wnioski z imputacji

Należy wziąć pod uwagę że RMSE liczone jest na całym zbiorze. Biorąc małe podzbiory danych błędy w kolejnych iteracjach bardzo się różniły, jednak używając większość zbioru były one prawie identyczne. Automatyczna imputacja jest szybkim rozwiązaniem, lecz wiążę się z dużym błędem oraz wysokim odchyleniem standardowym (duży rozrzut błędu). Im wiecej kolumn ma brakujące wartości, tym większe jest odchylenie standardowe tego błędu.